In [1]:
# 选择需要的因子
indicators = [
    "macd", "bbi", "ad", "ama", "pdi","mdi", 'mom',"sar",
    "kdj", "rsi", "roc", "bias", "cci", "osc", "cmo", "apo", 'ao',
     "bopa", "mfi", "wr", 'tema', 'vhf', 'cmf', 'vris', 'k', 'd', 'ri', 'cvi','obv']

rolling_window = 252
contract = 'T' #可换为T、TL、TF

### 数据清洗

In [2]:
'''
获取的基本数据， 和宏观经济指标, 在这里已经进行了ffill处理。
'''
import sys
sys.path.append("/Applications/Wind API.app/Contents/python")

from WindPy import *
import pandas as pd
import numpy as np
import talib as ta

def fetch_wind_data():
    # 启动WindPy接口
    ret = w.start()
    if not ret.ErrorCode == 0:
        raise Exception("WindPy启动失败")
    
    # 检查是否连接成功
    ret = w.isconnected()
    if not ret:
        raise Exception("WindPy未连接")

    # 提取TF和T的初始行情数据
    TF5 = w.wsd('TF.CFE', ['CLOSE', 'HIGH', 'LOW', 'OPEN', 'volume'], '2014-05-05', usedf=True)
    TF = TF5[1].copy()
    TF10 = w.wsd('T.CFE', ['CLOSE', 'HIGH', 'LOW', 'OPEN', 'volume'], '2015-03-20', usedf=True)
    T = TF10[1].copy()
    TL30 = w.wsd('TL.CFE', ['CLOSE', 'HIGH', 'LOW', 'OPEN', 'volume'], '2015-04-22', usedf=True)
    TL = TL30[1].copy()

    '''
    # 宏观数据，开始时间是2014-01-02，初始的缺失值用0填充
    ids = ["S0029657", "S0059749", "S0059744", "S0059747", "M0067855", "G0000886", "G0000889", "G0000891", 
           "G1306752", "G0006352", "G0006353", "M0000612", "M0001227", "M0074417", "M1004524", "M1004520", 
           "M0048486", "M0048488", "M0048490", "M0096868", "M0017142", "M0017141", "M0017145", "M1001854", 
           "S0181383", "S5808575", "S0031525", "M5525763", "M0041653", "M0041652",
           #新加的因子（27个）
          "M1004263","M1004267","M1004271","L4530250","U0737658","O8195887",
            "W6109272","U5267974","Y1667217","W1775339","U9659646","A0239140",
           "F2827408","Z6496161","Y4138099","M0041372","M0041374",
           "M0041378","M0329655","M1004899","M1004900","M1004902",'S0059745',
           'S0059752','M1004264','M1004274','W8696400']
    macro = w.edb(ids, beginTime="2014-01-02", ShowBlank=0, usedf=True)

    #将macro和行情数据拼接在一起
    macro_final = macro[1].copy()
    '''

    T = T.reset_index()
    T = T.rename(columns={'index': 'date'})
    TF= TF.reset_index()
    TF= TF.rename(columns={'index': 'date'})
    TL= TL.reset_index()
    TL= TL.rename(columns={'index': 'date'})   
    
    TF.columns = [col.lower() for col in TF.columns]
    T.columns = [col.lower() for col in T.columns]
    TL.columns = [col.lower() for col in TL.columns]

    '''
    #向前填充macro_final缺失值
    macro_final.replace(0, np.nan, inplace=True)  # Replace 0 with NA for proper forward fill
    macro_final.fillna(method='ffill', inplace=True)    
    macro_final = macro_final.fillna(macro_final.mean())
    macro_final= macro_final.reset_index()
    macro_final= macro_final.rename(columns={'index': 'date'})

    #分别合并TF,T
    merged_TF = pd.merge(TF, macro_final, on='date', how='left') 
    merged_T = pd.merge(T, macro_final, on='date', how='left')
    merged_TL = pd.merge(TL, macro_final, on='date', how='left')
    '''
    w.stop()
    return TF, T, TL

#------------------------------------------------------------------------------------
def moving_average_signal(data, short_window=5, long_window=20):
    data['ma5'] = data['close'].rolling(window=short_window).mean()
    data['ma20'] = data['close'].rolling(window=long_window).mean()
    data['ma_signal'] = np.where((data['ma5'] > data['ma20']), 1, 0)
    data['ma_signal'] = data['ma_signal'].ffill(limit=21).fillna(0)
    return data

def kdj_signal(data, window=9):
    data['rsv'] = (data['close'] - data['low'].rolling(window=window).min()) / \
                  (data['high'].rolling(window=window).max() - data['low'].rolling(window=window).min())
    data['k'] = data['rsv'].ewm(com=2).mean()
    data['d'] = data['k'].ewm(com=2).mean()
    data['kdj_signal'] = np.where(data['k'] > data['d'], 1, 0)
    return data

def pdmi_signal(data, window=10):
    data['pdi'] = ta.PLUS_DI(data['high'], data['low'], data['close'], timeperiod=window)
    data['mdi'] = ta.MINUS_DI(data['high'], data['low'], data['close'], timeperiod=window)
    data['pdmi_signal'] = np.where(data['pdi'] > data['mdi'], 1, 0)
    return data

def cci_signal(data, window=20):
    data['cci'] = ta.CCI(data['high'], data['low'], data['close'], timeperiod=window)
    data['cci_signal'] = np.where((data['cci'] <= data['cci'].rolling(90).quantile(0.3)), 1, \
                                  np.where(data['cci'] >= data['cci'].rolling(90).quantile(0.7) ,-1, 0))
    return data

def rsi_signal(data, fast_window=9, slow_window=14):
    data['rsi_fast'] = ta.RSI(data['close'], timeperiod=fast_window)
    data['rsi_slow'] = ta.RSI(data['close'], timeperiod=slow_window)
    data['rsi_signal'] = np.where((data['rsi_fast'] > data['rsi_slow']), 1, 0)
    return data

def macd_signal(data):
    data['macd'], data['macd_signal'], _ = ta.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    return data
    
# ==================== BOP (多空平衡指标) ====================
def bop_signal(data):
    data['bop'] = ta.BOP(data['open'], data['high'], data['low'], data['close'])
    # BOP上穿0轴时产生买入信号
    data['bop_signal'] = np.where(data['bop'] > 0, 1, 0)
    return data

# ==================== AROON (阿隆指标) ====================
def aroon_signal(data, up=70):
    aroon_down, aroon_up = ta.AROON(data['high'], data['low'], timeperiod=14)
    data['aroon_up'] = aroon_up
    data['aroon_down'] = aroon_down
    # 当阿隆上行线超过up且高于下行线时触发信号
    data['aroon_signal'] = np.where(
        (aroon_up > up) & (aroon_up > aroon_down), 1, 0
    )
    return data

# ==================== Williams %R (威廉超买超卖指标) ====================
def wr_signal(data):
    data['wr'] = ta.WILLR(data['high'], data['low'], data['close'], timeperiod=10)
    # 判断过冷和过热
    data['wr_signal'] = np.where((data['wr'] <= data['wr'].rolling(252).quantile(0.3)), 1, \
                                  np.where(data['wr'] >= data['wr'].rolling(252).quantile(0.7) ,-1, 0))
    return data

# ==================== TRIX (三重指数平均线) ====================
def trix_signal(data, period1=10, period2=3):
    data['trix'] = ta.TRIX(data['close'], timeperiod=period1)
    # TRIX线上穿信号线（3日均线）时买入
    data['trix_signal'] = np.where(
        data['trix'] > data['trix'].rolling(period2).mean(), 1, 0
    )
    return data

def dma_signal(data, short_period=10, long_period=50):
    data = data.copy()
    data['dma_short'] = data['close'].rolling(short_period).mean()
    data['dma_long'] = data['close'].rolling(long_period).mean()
    data['dma'] = data['dma_short'] - data['dma_long']
    
    # 生成信号：dma 上穿 0 → 买入，dma 下穿 0 → 卖出
    data['dma_signal'] = np.where((data['dma'] > 0), 1,
                           np.where((data['dma'] < 0), -1, 0))
    return data

# ==================== ATR 波动率指标 ====================
def atr_signal(data, window=10):
    """
    计算 ATR（平均真实波动幅度），用于衡量市场波动性。
    若 ATR 高于历史 60 日的 75% 分位数，则认为当前波动较大 → 卖出信号。
    """
    data['atr'] = ta.ATR(data['high'], data['low'], data['close'], timeperiod=window)
    data['atr_signal'] = np.where(data['atr'] >= data['atr'].rolling(60).quantile(0.6), 1,\
                                   np.where(data['atr'] <= data['atr'].rolling(60).quantile(0.4), -1, 0))
    return data

# ==================== OBV 能量潮指标 ====================
def obv_signal(data):
    """
    OBV（On-Balance Volume）表示成交量与价格趋势的同步性。
    OBV 上升表示资金流入，生成买入信号。
    """
    data['obv'] = ta.OBV(data['close'], data['volume'])
    data['obv_signal'] = np.where(data['obv'] > data['obv'].shift(1), 1, -1)
    return data

# ==================== 布林带宽度信号 ====================
def bb_width_signal(data, window=10):
    """
    计算布林带宽度作为波动性指标。
    过冷过热信号判断。
    """
    data['bb_width'] = data['close'].rolling(window).std()
    data['bb_width_signal'] = np.where(data['bb_width'] > data['bb_width'].rolling(60).quantile(0.7), -1,\
                                        np.where(data['bb_width'] < data['bb_width'].rolling(60).quantile(0.3), 1, 0))
    return data

# ==================== CMO 钱德动量摆动指标 ====================
def cmo_signal(data, window=30):
    """
    CMO 反映了价格的动量强弱。
    过冷过热信号。
    """
    data['cmo'] = ta.CMO(data['close'], timeperiod=14)
    data['cmo_signal'] = np.where(
        data['cmo'] > data['cmo'].rolling(window).quantile(0.7), -1, np.where(
        data['cmo'] < data['cmo'].rolling(window).quantile(0.3), 1, 0)
    )
    return data

# ==================== 均值回归信号（Z-Score） ====================
def mean_reversion_signal(data, window=10, threshold=1):
    """
    计算价格的 Z-Score，用于均值回归策略。
    当价格低于均值 2 个标准差以下时（Z < -2），认为价格超跌 → 买入；
    当 Z > 2 时，价格高估 → 不持仓。
    """
    data['z_score'] = (data['close'] - data['close'].rolling(window).mean()) / data['close'].rolling(window).std()
    data['mean_reversion_signal'] = np.where(data['z_score'] > threshold, 1,
                                              np.where(data['z_score'] < -threshold, -1, 0))
    return data

# ==================== ROC 变动率信号 ====================
def roc_signal(data, period=12):
    """
    ROC（Rate of Change）衡量当前价格相较于 n 日前价格的变化速度。
    ROC 上穿 0 → 买入；ROC 下穿 0 → 卖出。
    持仓状态以1/0形式呈现。
    """
    data = data.copy()
    data['roc'] = (data['close'] - data['close'].shift(period)) / data['close'].shift(period)
    data['roc_signal'] = np.where(data['roc'] > 0, 1, -1)
    return data
#----------------------------------------------------------------------------

def mom(data, period=10): # 返回的即为指标因子值
    return data['close'] - data['close'].shift(period)   


def cmo(data, period=10): # 返回的即为指标因子值
    delta = data['close'].diff()
    up = delta.apply(lambda x: max(x, 0))  # Up_t
    dn = delta.apply(lambda x: min(x, 0))  # Dn_t

    sum_up = up.rolling(window=period, min_periods=1).sum()
    sum_dn = abs(dn.rolling(window=period, min_periods=1).sum())

    cmo_value = 100 * (sum_up - sum_dn) / (sum_up + sum_dn)
    return cmo_value


def apo(data, short_period=10, long_period=20): # 返回的即为指标因子值
    ma_short = data['close'].rolling(window=short_period).mean()
    ma_long = data['close'].rolling(window=long_period).mean()
    return ma_short - ma_long


def macd(data, short_period=10, long_period=20, signal_period=9):  # 只返回了 hist，用来作为因子值，判断过冷过热等
    def ema(data, period):
        ema = [0] * len(data)  # 初始化 EMA 数组
        multiplier = 2 / (period + 1)
        for i in range(len(data)):
            if i == 0:
                ema[i] = data[i]
            else:
                ema[i] = ((data[i] - ema[i - 1]) * multiplier) + ema[i - 1]
        return ema

    ema_short = ema(data['close'].tolist(), short_period)
    ema_long = ema(data['close'].tolist(), long_period)

    dif = [s - l for s, l in zip(ema_short, ema_long)]
    dea = ema(dif, signal_period)
    hist = [2 * (d - e) for d, e in zip(dif, dea)]

    return hist

def dmi(data, period=10):    # 需要修改因子值，从而去适应后面判断市场状态
    high_diff = data['high'].diff()
    low_diff = -data['low'].diff()
    
    # +DM 和 -DM
    plus_dm = high_diff.where((high_diff > 0) & (high_diff > low_diff), 0.0)
    minus_dm = low_diff.where((low_diff > 0) & (low_diff > high_diff), 0.0)
    
    # 真实波幅 TR
    tr1 = data['high'] - data['low']
    tr2 = abs(data['high'] - data['close'].shift(1))
    tr3 = abs(data['low'] - data['close'].shift(1))
    tr = tr1.combine(tr2, max).combine(tr3, max)
    
    # +DI 和 -DI
    tr_sum = tr.rolling(window=period, min_periods=1).sum()
    plus_di = 100 * (plus_dm.rolling(window=period, min_periods=1).sum() / tr_sum)
    minus_di = 100 * (minus_dm.rolling(window=period, min_periods=1).sum() / tr_sum)
    
    # DX
    dx = 100 * abs((plus_di - minus_di) / (plus_di + minus_di))
    
    # ADX
    adx = dx.rolling(window=period, min_periods=1).mean()
    
    return plus_di, minus_di, adx


def cci(data, period=10):
    tp = (data['high'] + data['low'] + data['close']) / 3
    sma_tp = tp.rolling(window=period, min_periods=1).mean()
    mean_deviation = tp.rolling(window=period).apply(lambda x: np.mean(np.abs(x - np.mean(x))), raw=True)
    cci_value = (tp - sma_tp) / (0.015 * mean_deviation)
    return cci_value


def bopa(data, period=20):
    def bop(data):
        return (data['open'] - data['close']) / (data['high'] - data['low'])
    
    bop_values = bop(data)
    return bop_values.rolling(window=period).mean()


def mfi(data, period=10):
    typical_price = (data['high'] + data['low'] + data['close']) / 3
    raw_money_flow = typical_price * data['volume']
    
    positive_flow = []
    negative_flow = []
    
    for i in range(1, len(data)):
        if typical_price[i] > typical_price[i - 1]:
            positive_flow.append(raw_money_flow[i])
            negative_flow.append(0)
        elif typical_price[i] < typical_price[i - 1]:
            positive_flow.append(0)
            negative_flow.append(raw_money_flow[i])
        else:
            positive_flow.append(0)
            negative_flow.append(0)

    positive_flow = pd.Series(positive_flow).rolling(window=period).sum()
    negative_flow = pd.Series(negative_flow).rolling(window=period).sum()

    money_flow_ratio = positive_flow / negative_flow
    mfi_value = 100 - (100 / (1 + money_flow_ratio))
    
    # 为保持与原始数据对齐，向前填充第一个周期的 NaN 值
    mfi_value = pd.Series([None] + mfi_value.tolist())

    return mfi_value

def aroon(data, period=25):
    aroon_up = 100 * ((period - (period - (data['high'].rolling(window=period).apply(lambda x: list(x).index(max(x)), raw=True)))) / period)
    aroon_down = 100 * ((period - (period - (data['low'].rolling(window=period).apply(lambda x: list(x).index(min(x)), raw=True)))) / period)
    return aroon_up-aroon_down

def bbi(data):
    ma3 = data['close'].rolling(window=3).mean()
    ma6 = data['close'].rolling(window=6).mean()
    ma12 = data['close'].rolling(window=12).mean()
    ma24 = data['close'].rolling(window=24).mean()
    bbi_value = (ma3 + ma6 + ma12 + ma24) / 4
    return bbi_value

def bbi_sig(data):
    bbi_val = data['bbi']
    bbi_sig_value = (bbi_val - bbi_val.shift(1)) / bbi_val.shift(1)
    return bbi_sig_value

# 计算 TR, DMZ, 和 DMF
def dmz_dmf(data):
    tr = data['high'] - data['low']
    dmz = tr.where((data['high'] + data['low']) > (data['high'].shift(1) + data['low'].shift(1)), 0)
    dmf = tr.where((data['high'] + data['low']) < (data['high'].shift(1) + data['low'].shift(1)), 0)
    return tr, dmz, dmf

# 计算 DIZ 和 DIF
def diz_dif(data, period=10):
    tr, dmz, dmf = dmz_dmf(data)
    sum_dmz = dmz.rolling(window=period, min_periods=1).sum()
    sum_dmf = dmf.rolling(window=period, min_periods=1).sum()
    sum_tr = sum_dmz + sum_dmf
    diz = sum_dmz / sum_tr
    dif = sum_dmf / sum_tr
    return diz, dif

# 计算 DDI
def ddi(data, period_dif=10, period_diz=10, period_ddi=10):
    diz, dif = diz_dif(data, period_diz)
    ddi_val = diz - dif
    ddi_sma = ddi_val.rolling(window=period_ddi, min_periods=1).mean()
    return ddi_val

# 计算 ADD 指标
def add(data, period_add=10):
    ddi_vals = data['ddi'].tolist()
    add_vals = []
    for t in range(len(ddi_vals)):
        if t < period_add - 1:
            add_vals.append(None)
        else:
            numerator = sum((11 - i) * ddi_vals[t + 1 - i] for i in range(1, 11))
            denominator = sum(range(1, 11))
            add_vals.append(numerator / denominator)
    return pd.Series(add_vals)

# 计算 AD 指标
def ad(data, period_ad=5):
    add_vals = data['add'].rolling(window=period_ad, min_periods=1).sum()
    return add_vals / period_ad

#  计算 DMA
def dma(data, short_period=10, long_period=20, ama_period=10):
    # 计算短期和长期移动平均线
    ma_short = data['close'].rolling(window=short_period).mean()
    ma_long = data['close'].rolling(window=long_period).mean()
    
    # 计算 DIF
    dif = ma_short - ma_long
    
    # 计算 AMA
    ama = dif.rolling(window=ama_period).mean()
    
    return dif,ama

# 计算 SAR
def sar(data, initial_af=0.02, step_af=0.02, max_af=0.2, accel_period=5):
    sar = [0] * len(data)
    af = initial_af
    uptrend = True

    # 初始化 SAR 和 EP
    ep = data['low'][0] if uptrend else data['high'][0]
    sar[0] = data['low'][0] if uptrend else data['high'][0]
    
    for i in range(1, len(data)):
        prev_sar = sar[i - 1]
        prev_ep = ep
        prev_af = af

        # 判断条件
        if i >= 2 * accel_period:
            cond1 = (i >= accel_period and
                 data['close'][i] > max(data['close'][i - accel_period], data['close'][i - 2 * accel_period]) and
                 data['high'][i] > data['high'][i - 1])
            cond2 = (i >= accel_period and
                 data['close'][i] < min(data['close'][i - accel_period], data['close'][i - 2 * accel_period]) and
                 data['low'][i] < data['low'][i - 1])
        else:
            cond1 = cond2 = False

        # 更新加速因子 AF
        if uptrend:
            if cond1:
                af = min(max_af, prev_af + step_af)
            else:
                af = 0.02
        else:
            if cond2:
                af = min(max_af, prev_af + step_af)
            else:
                af = 0.02

        # 更新极值点 EP
        if uptrend:
            ep = max(prev_ep, data['high'][i])
        else:
            ep = min(prev_ep, data['low'][i])

        # 计算当前 SAR
        sar[i] = prev_sar + prev_af * (ep - prev_sar)

        # 判断反转
        if uptrend:
            if data['low'][i] < sar[i]:
                uptrend = False
                sar[i] = prev_ep
                ep = data['low'][i]
                af = initial_af
            else:
                ep = max(ep, data['high'][i])
        else:
            if data['high'][i] > sar[i]:
                uptrend = True
                sar[i] = prev_ep
                ep = data['high'][i]
                af = initial_af
            else:
                ep = min(ep, data['low'][i])

        # 约束 SAR 的最终值
        if i >= accel_period:
            if uptrend:
                sar[i] = min(sar[i], min(data['low'][i - accel_period:i + 1]))
            else:
                sar[i] = max(sar[i], max(data['high'][i - accel_period:i + 1]))

    return pd.Series(sar)


def trix_and_trma(data):
    close_prices = data['close']
    ax_values = np.zeros(len(close_prices))
    bx_values = np.zeros(len(close_prices))
    trix_values = np.zeros(len(close_prices))
    trma_values = np.zeros(len(close_prices))
    
    # Calculate AX_T
    for t in range(len(close_prices)):
        if t == 0:
            ax_values[t] = 0
        elif t == 1:
            ax_values[t] = (9/11) * close_prices[t-1] + (2/11) * close_prices[t]
        else:
            ax_values[t] = (9/11) * ax_values[t-1] + (2/11) * close_prices[t]
    
    # Calculate BX_T
    for t in range(len(close_prices)):
        if t == 0 or t == 1:
            bx_values[t] = 0
        elif t == 2:
            bx_values[t] = (9/11) * ax_values[0] + (2/11) * ax_values[1]
        else:
            bx_values[t] = (9/11) * bx_values[t-1] + (2/11) * ax_values[t]
    
    # Calculate TRIX_T
    for t in range(len(close_prices)):
        if t < 3:
            trix_values[t] = 0
        elif t == 3:
            trix_values[t] = (9/11) * bx_values[0] + (2/11) * bx_values[1]
        else:
            trix_values[t] = (9/11) * trix_values[t-1] + (2/11) * bx_values[t]
    
    # Calculate TRMA_T
    for t in range(len(close_prices)):
        if t < 9:
            trma_values[t] = 0
        else:
            trma_values[t] = close_prices[t] - np.sum(trix_values[t-9:t+1]) / 10
    
    trix_series = pd.Series(trix_values, index=data.index)
    trma_series = pd.Series(trma_values, index=data.index)
    
    return trix_series, trma_series

def kdj(data, period=9):
    # 计算 RSV
    low_min = data['low'].rolling(window=period, min_periods=1).min()
    high_max = data['high'].rolling(window=period, min_periods=1).max()
    rsv = (data['close'] - low_min) / (high_max - low_min) * 100

    # 初始化 %K 和 %D
    k = [50] * len(rsv)
    d = [50] * len(rsv)

    # 计算 %K 和 %D
    for i in range(1, len(rsv)):
        k[i] = (2 / 3) * k[i - 1] + (1 / 3) * rsv[i]
        d[i] = (2 / 3) * d[i - 1] + (1 / 3) * k[i]

    # 计算 %J
    j = [3 * k[i] - 2 * d[i] for i in range(len(k))]

    return  pd.Series(j)

def rsi(data, period=10):
    # 计算每日价格变化
    delta = data['close'].diff()

    # 将涨幅和跌幅分开计算
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # 计算平均涨幅和平均跌幅
    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()

    # 计算 RS 和 RSI
    rs = avg_gain / avg_loss
    rsi_value = 100 - (100 / (1 + rs))

    return rsi_value

def roc(data, period=5):
    n_period_ago = data['close'].shift(period)
    roc_value = (data['close'] - n_period_ago) / n_period_ago *100
    return roc_value

def maroc(data, roc_values, period=10):
    """计算 MAROC 指标"""
    return roc_values.rolling(window=period, min_periods=1).mean()

def bias(data, period=10):
    """计算 BIAS 指标"""
    moving_average = data['close'].rolling(window=period, min_periods=1).mean()
    bias_value = (data['close'] - moving_average) / moving_average * 100
    return bias_value


def osc(data, period=10):
    moving_average = data['close'].rolling(window=period, min_periods=1).mean()
    osc_value = data['close'] - moving_average
    return osc_value

def oscma1(osc_values, period=10):
    weights = [11 - i for i in range(1, period + 1)]
    weight_sum = sum(range(1, period + 1))
    oscma_values = [0] * len(osc_values)

    for t in range(period - 1, len(osc_values)):
        weighted_sum = sum(weights[i] * osc_values[t + 1 - period + i] for i in range(period))
        oscma_values[t] = 100 * weighted_sum / weight_sum

    return oscma_values

def wr(data, period=9):
    highest_high = data['high'].rolling(window=period, min_periods=1).max()
    lowest_low = data['low'].rolling(window=period, min_periods=1).min()
    wr_value = (highest_high - data['close']) / (highest_high - lowest_low)*100 
    return wr_value

def ema(data, period):
    return data.ewm(span=period, adjust=False).mean()

def tema(data, period=9):
    ema1 = ema(data['close'], period)
    ema2 = ema(ema1, period)
    ema3 = ema(ema2, period)
    return 3 * ema1 - 3 * ema2 + ema3

def vhf(data, period=28):
    high_max = data['high'].rolling(window=period).max()
    low_min = data['low'].rolling(window=period).min()
    close_diff = abs(data['close'] - data['close'].shift(period))
    close_diff = close_diff.replace(0, 1e-6)     # Avoid division by zero
    return (high_max - low_min) / close_diff

def cmf(data, period=20):
    mfv = ((data['close'] - data['low']) - (data['high'] - data['close'])) / (data['high'] - data['low']) * data['volume']
    return mfv.rolling(window=period).sum() / data['volume'].rolling(window=period).sum()

def vris(data, period=12):
    return (data['volume'] - data['volume'].shift(period)) / data['volume'].shift(period) * 100

def stochastic_k(data, period=14):
    low_min = data['low'].rolling(window=period).min()
    high_max = data['high'].rolling(window=period).max()
    return 100 * (data['close'] - low_min) / (high_max - low_min)

def stochastic_d(data, k_period=14, d_period=3):
    k = stochastic_k(data, k_period)
    return k.rolling(window=d_period).mean()

def relative_index(data, period=14):
    delta = data['close'].diff()
    gain = delta.where(delta > 0, 0.0).rolling(window=period).sum()
    loss = -delta.where(delta < 0, 0.0).rolling(window=period).sum()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def cvi(data, period=20):
    daily_volatility = (data['high'] - data['low']) / data['close']
    return daily_volatility.rolling(window=period).sum()

def obv(data):
    obv = ta.OBV(data['close'], data['volume'])
    return obv

def bb_width(data, window=10):
    bb_width = data['close'].rolling(window).std()
    return bb_width

def get_indicator(data):
    data = moving_average_signal(data)
    data = kdj_signal(data)
    data = pdmi_signal(data)
    data = cci_signal(data)
    data = rsi_signal(data)
    data = macd_signal(data)
    data = bop_signal(data)
    data = aroon_signal(data)
    data = wr_signal(data)
    data = trix_signal(data)
    data = dma_signal(data)

    data = atr_signal(data)
    data = obv_signal(data)
    data = bb_width_signal(data)
    data = cmo_signal(data)
    data = mean_reversion_signal(data)
    data = roc_signal(data)
    
    return data


def calculate_indicators(data):
    data = data.copy()
    # 计算 10 天动量
    data['mom'] = mom(data, period=10)    

    # 计算 10 天 CMO
    data['cmo'] = cmo(data, period=10)

    # 计算 MA10 - MA20 的 APO
    data['apo'] = apo(data, short_period=10, long_period=20)

    #计算 MACD (MA10 - MA20) 并生成 DIF, DEA 和 HIST
    data['macd']=macd(data)

    # 计算 10天 DMI (其实是两个指标  PDI 和 MDI  ADX和ADXR )
    data['pdi'], data['mdi'], data['adx'] = dmi(data, period=10)     
    # data['adxr'] = (data['adx'] + data['adx'].shift(9)) / 2   # 补充了一个ADXR

    #计算 10 天 CCI
    data['cci'] = cci(data, period=10)

    # 使用 BOP 的 20 天均值生成 BOPA
    data['bopa'] = bopa(data, period=20)

    # 计算 10 天 MFI
    data['mfi'] = mfi(data, period=10)

    # 计算25天的AROON
    data['ao'] = aroon(data, period=25)

    # 第二篇研报的补充的11个因子

    # 计算 BBI
    data['bbi'] = bbi(data)
    data['bbi'] = bbi_sig(data)

    # 计算DDI
    data['ddi'] = ddi(data, period_dif=10, period_diz=10, period_ddi=10)
    data['add'] = add(data, period_add=10)
    data['ad'] = ad(data, period_ad=5)

    # 计算 DMA
    data['dma'], data['ama']= dma(data, short_period=10, long_period=20, ama_period=10)

    # 计算 SAR
    data['sar'] = sar(data)

    # 计算 TRIX 和 TRMA
    data['trix'], data['trma']= trix_and_trma(data)

    #计算 KDJ
    data['kdj'] = kdj(data, period=9)

    #计算 RSI
    data['rsi'] = rsi(data, period=10)

    # 10 天 ROC 和 MAROC
    data['roc'] = roc(data, period=10)
    # data['maroc'] = maroc(data, data['roc'], period=10)

    #计算 10 天 BIAS
    data['bias'] = bias(data, period=10)

    #计算 10 天 OSC
    data['osc']= osc(data, period=10)
    # data['oscma']=oscma1(data['osc'],period=10)

    # 计算 9天 WR
    data['wr'] = wr(data, period=9)

    # 额外补充的因子
    data['tema'] = tema(data)
    data['vhf'] = vhf(data)
    data['cmf'] = cmf(data)
    data['vris'] = vris(data)
    data['k'] = stochastic_k(data)
    data['d'] = stochastic_d(data)
    data['ri'] = relative_index(data)
    data['cvi'] = cvi(data)
    data['obv'] = obv(data)
    data['bb_width'] = bb_width(data)
    return data


TF_data, T_data, TL_data = fetch_wind_data()

#TF.to_csv("Users/huanghaotian/intern/银河证券/dataset/TF.csv",index=False)
#T.to_csv("Users/huanghaotian/intern/银河证券/dataset/T.csv",index=False)
#TL.to_csv("Users/huanghaotian/intern/银河证券/dataset/TL.csv",index=False)

TF_data.dropna(inplace=True)
TF_data.reset_index(drop=True,inplace=True)
T_data.dropna(inplace=True)
T_data.reset_index(drop=True,inplace=True)
TL_data.dropna(inplace=True)
TL_data.reset_index(drop=True,inplace=True)


TL_signal = get_indicator(TL_data.copy()).dropna()
T_signal = get_indicator(T_data.copy()).dropna()
TF_signal = get_indicator(TF_data.copy()).dropna()

TF_data['date'] = pd.to_datetime(TF_data['date'])
T_data['date'] = pd.to_datetime(T_data['date'])
TL_data['date'] = pd.to_datetime(TL_data['date'])

T_indicators = calculate_indicators(T_data)
TL_indicators = calculate_indicators(TL_data)
TF_indicators = calculate_indicators(TF_data)

if contract=='T':
    indicators_data = T_indicators
    signal_data = T_signal
elif contract=='TF':
    indicators_data = TF_indicators
    signal_data = TF_signal
if contract=='TL':
    indicators_data = TL_indicators
    signal_data = TL_signal

indicators_data['date'] = pd.to_datetime(indicators_data['date'])
signal_data['date'] = pd.to_datetime(signal_data['date'])

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [3]:
# 滚动窗口更新信号并应用
import os
# 设置文件路径
file_path = f'signal_data_{contract}.csv'
    
# 将结果保存到 CSV 文件
signal_data.to_csv(file_path, index=False)
print(f"信号已生成并保存为 {file_path}")

信号已生成并保存为 signal_data_T.csv


In [4]:
import pandas as pd
import numpy as np

# 用于计算初始阈值
def calculate_initial_thresholds(df, indicator):  
    quantiles = df[indicator].quantile([0.3, 0.4, 0.6, 0.7])

    thresholds = {
        'overheat': quantiles[0.7],  # 过热
        'cool': quantiles[0.3],   # 过冷
        'warming_lower': quantiles[0.4], # 转暖 低
        'warming_upper': quantiles[0.6], # 转暖 高
        'cool_lower': quantiles[0.4], # 转冷 低
        'cool_upper': quantiles[0.6]  # 转冷 高
    }
    return thresholds

# 用于优化阈值
def optimize_thresholds(df, indicator, thresholds, window=10): 
    """
    根据市场状态的条件优化阈值。通过遍历百分位数来调节和选择最佳阈值。
    主要优化条件包括发生概率、上涨概率、涨跌比等。

    参数：
    df : DataFrame - 包含历史数据的DataFrame。
    indicator : str - 用于优化阈值的技术指标列名称。
    thresholds : dict - 初始的市场状态阈值。
    window : int - 用于计算未来涨跌的时间窗口。

    返回：
    dict - 优化后的市场状态阈值。
    """
    results = {}
    percentiles = np.arange(0, 101, 1)
    df = df.copy()


    # 限定阈值选取范围
    median = df[indicator].median()
    std = df[indicator].std()
    df = df[(df[indicator] >= median - 3 * std) & (df[indicator] <= median + 3 * std)]#筛除异常值


    # Overheat (Sell)  市场过热，考虑做空
    best_threshold = None
    best_metric = np.inf
    best_up_prob=None
    best_ratio=None
    best_occurrence_prob=None

    for p in percentiles: # 百分比遍历调优
        threshold = np.percentile(df[indicator], p)

        # 条件1：分位数区间
        if threshold < thresholds['overheat']: # 如果小于过热阈值，跳过
            continue 

        #  条件2：发生概率
        df['temp_signal'] = 0 
        df.loc[(df[indicator] > threshold) & (df[indicator].shift(1) <= threshold), 'temp_signal'] = -1#今天相比昨天上穿threshold

        subset = df[df['temp_signal'] == -1]
        occurrence_prob = len(subset) / len(df)
        if occurrence_prob <= 0.01:  # 发生概率小于1%，跳过；控制过热的概率大于1%
            continue

        # 条件3：上涨概率
        future_returns = (df['close'].shift(-window) - df['close']) / df['close']
        future_returns = future_returns.loc[subset.index]
        up_prob = (future_returns > 0).sum() / len(future_returns)

        if up_prob > 0.5:  # 上涨概率大于50%，跳过；检查当前信号发生后，未来价格上涨的概率是否低于 50%（即市场过热时不应继续上涨）。
            continue

        # 条件4：涨跌比
        down_prob=(future_returns < 0).sum() / len(future_returns)

        up_mean = future_returns[future_returns > 0].mean()
        down_mean = future_returns[future_returns < 0].mean()
        ratio=up_prob * up_mean / ((abs(down_mean) if down_mean != 0 else 1)*(abs(down_prob) if down_prob != 0 else 1))
        if ratio>1:   # 涨跌比大于1，跳过；通过比较上涨和下跌的比率，来确认市场是否真的处于过热状态。
            continue

        # 对阈值进行优化
        metric = ratio*up_prob

        if metric < best_metric: # 过热是做空，取上涨概率和涨跌比之积最小的值作为卖出阈值
            best_metric = metric
            best_up_prob=up_prob
            best_ratio=ratio
            best_threshold = threshold
            best_occurrence_prob=occurrence_prob
        else:
            continue

    results['overheat'] = {
        'threshold': best_threshold,
        'up_prob':best_up_prob,
        'ratio':best_ratio,
        'metric': best_metric,
        'occurrence_prob': best_occurrence_prob
    }


    # cool (Buy) 市场过冷，考虑做多
    best_threshold = None
    best_metric = -np.inf
    best_up_prob=None
    best_ratio=None
    best_occurrence_prob=None

    for p in percentiles:

        threshold = np.percentile(df[indicator], p)
        if threshold > thresholds['cool']:
            continue

        df['temp_signal'] = 0
        df.loc[(df[indicator] < threshold) & (df[indicator].shift(1) >= threshold), 'temp_signal'] = 1
        subset = df[df['temp_signal'] == 1]
        occurrence_prob = len(subset) / len(df)
        if occurrence_prob <= 0.01:
            continue

        # 条件3：上涨概率
        future_returns = (df['close'].shift(-window) - df['close']) / df['close']
        future_returns = future_returns.loc[subset.index]
        up_prob = (future_returns > 0).sum() / len(future_returns)

        if up_prob < 0.5:  # 上涨概率小于50%，跳过
            continue

        # 条件4：涨跌比
        down_prob=(future_returns < 0).sum() / len(future_returns)

        up_mean = future_returns[future_returns > 0].mean()
        down_mean = future_returns[future_returns < 0].mean()
        ratio=up_prob * up_mean / ((abs(down_mean) if down_mean != 0 else 1)*(abs(down_prob) if down_prob != 0 else 1))
        if ratio<1:   # 涨跌比小于1，跳过
            continue

        # 对阈值进行优化
        metric = ratio*up_prob

        if metric > best_metric: # 过冷是做多，取上涨概率和涨跌比之积最大的值作为买入阈值
            best_metric = metric
            best_up_prob=up_prob
            best_ratio=ratio
            best_threshold = threshold
            best_occurrence_prob=occurrence_prob
        else:
            continue

    results['cool'] = {
        'threshold': best_threshold,
        'up_prob':best_up_prob,
        'ratio':best_ratio,
        'metric': best_metric,
        'occurrence_prob': best_occurrence_prob
    }

    # Turn-Warming (Buy) 市场转暖，考虑做多
    best_threshold = None
    best_metric = -np.inf
    best_up_prob=None
    best_ratio=None
    best_occurrence_prob=None
    
    for p in percentiles:
        threshold = np.percentile(df[indicator], p)
        
        if threshold < thresholds['warming_lower'] or threshold > thresholds['warming_upper']:
            continue

        df['temp_signal'] = 0
        df.loc[(df[indicator] <= thresholds['warming_upper'])&(df[indicator] > threshold) & (df[indicator].shift(1) < df[indicator]), 'temp_signal'] = 1

        subset = df[df['temp_signal'] == 1]
        occurrence_prob = len(subset) / len(df)
        if occurrence_prob <= 0.01:
            continue

        # 条件3：上涨概率
        future_returns = (df['close'].shift(-window) - df['close']) / df['close']
        future_returns = future_returns.loc[subset.index]
        up_prob = (future_returns > 0).sum() / len(future_returns)

        if up_prob < 0.5:  # 上涨概率小于50%，跳过
            continue

        # 条件4：涨跌比
        down_prob=(future_returns < 0).sum() / len(future_returns)

        up_mean = future_returns[future_returns > 0].mean()
        down_mean = future_returns[future_returns < 0].mean()
        ratio=up_prob * up_mean / ((abs(down_mean) if down_mean != 0 else 1)*(abs(down_prob) if down_prob != 0 else 1))
        if ratio<1:   # 涨跌比小于1，跳过
            continue

        # 对阈值进行优化
        metric = ratio*up_prob

        if metric > best_metric: # 转暖是做多，取上涨概率和涨跌比之积最大的值作为买入阈值
            best_metric = metric
            best_up_prob=up_prob
            best_ratio=ratio
            best_threshold = threshold
            best_occurrence_prob=occurrence_prob
        else:
            continue

    results['turn_warm'] = {
        'threshold': best_threshold,
        'up_prob':best_up_prob,
        'ratio':best_ratio,
        'metric': best_metric,
        'occurrence_prob': best_occurrence_prob
    }

    # Turning Cool (Sell)
    best_threshold = None
    best_metric = np.inf
    best_up_prob=None
    best_ratio=None
    best_occurrence_prob=None

    
    for p in percentiles:
        threshold = np.percentile(df[indicator], p)
        if threshold < thresholds['cool_lower'] or threshold > thresholds['cool_upper']:
            continue

        df['temp_signal'] = 0
        df.loc[(df[indicator] >= thresholds['cool_lower'])&(df[indicator] < threshold) & (df[indicator].shift(1) > df[indicator]), 'temp_signal'] = -1
        subset = df[df['temp_signal'] == -1]
        occurrence_prob = len(subset) / len(df)
        if occurrence_prob <= 0.01:
            continue

        # 条件3：上涨概率
        future_returns = (df['close'].shift(-window) - df['close']) / df['close']
        future_returns = future_returns.loc[subset.index]
        up_prob = (future_returns > 0).sum() / len(future_returns)

        if up_prob > 0.5:  # 上涨概率大于50%，跳过
            continue

        # 条件4：涨跌比
        down_prob=(future_returns < 0).sum() / len(future_returns)

        up_mean = future_returns[future_returns > 0].mean()
        down_mean = future_returns[future_returns < 0].mean()
        ratio=up_prob * up_mean / ((abs(down_mean) if down_mean != 0 else 1)*(abs(down_prob) if down_prob != 0 else 1))
        if ratio>1:   # 涨跌比大于1，跳过
            continue

        # 对阈值进行优化
        metric = ratio*up_prob

        if metric < best_metric: # 转冷是做空，取上涨概率和涨跌比之积最小的值作为卖出阈值
            best_metric = metric
            best_up_prob=up_prob
            best_ratio=ratio
            best_threshold = threshold
            best_occurrence_prob=occurrence_prob
        else:
            continue

    results['turn_cool'] = {
        'threshold': best_threshold,
        'up_prob':best_up_prob,
        'ratio':best_ratio,
        'metric': best_metric,
        'occurrence_prob': best_occurrence_prob
    }

    return results

# 用于生成信号
def generate_signals(df, indicator, results, thresholds):
    df = df.copy()
    df.reset_index(drop=True, inplace=True)
    df['signal'] = 0
    df['condition'] = 'neutral'  # 默认处于中性状态

    # Process 'overheat'
    if results['overheat']['threshold'] is not None:
        threshold = results['overheat']['threshold']
        df.loc[(df[indicator] > threshold), 'signal'] = -1
        df.loc[(df[indicator] > threshold), 'condition'] = 'overheat'

    # Process 'cool'
    if results['cool']['threshold'] is not None:
        threshold = results['cool']['threshold']
        df.loc[(df[indicator] < threshold), 'signal'] = 1
        df.loc[(df[indicator] < threshold), 'condition'] = 'cool'

    # Process 'turn_warm'
    if results['turn_warm']['threshold'] is not None:
        threshold_warm = results['turn_warm']['threshold']
        threshold_overheat = results['overheat']['threshold']
        df.loc[(df[indicator] < threshold_overheat) & 
               (df[indicator] >= threshold_warm), 'signal'] = 1
        df.loc[(df[indicator] < threshold_overheat) & 
               (df[indicator] >= threshold_warm), 'condition'] = 'turn_warm'

    # Process 'turn_cool'
    if results['turn_cool']['threshold'] is not None:
        threshold_cool = results['cool']['threshold']
        threshold_turn_cool = results['turn_cool']['threshold']
        df.loc[(df[indicator] > threshold_cool) & 
               (df[indicator] <= threshold_turn_cool), 'signal'] = -1
        df.loc[(df[indicator] > threshold_cool) & 
               (df[indicator] <= threshold_turn_cool), 'condition'] = 'turn_cool'

    # Generate continuous signals
    df['final_signal'] = df['signal'].copy()
    last_signal = 0
    last_condition = 'neutral'
    started = False

    for i in range(len(df)):
        if df.at[i, 'signal'] != 0:
            last_signal = df.at[i, 'signal']
            last_condition = df.at[i, 'condition']
            started = True
        elif started:
            df.at[i, 'final_signal'] = last_signal
            df.at[i, 'condition'] = last_condition

    return df

# 使用滚动窗口更新阈值和生成信号
def rolling_threshold_update(df, indicators, window=252):
    updated_df = df.copy()

    for indicator in indicators:    
        for i in range(window, len(df)):
            if i % window==0:
                rolling_data = df.iloc[i - window:i].copy()
                thresholds_st = calculate_initial_thresholds(rolling_data, indicator)
                results = optimize_thresholds(rolling_data, indicator, thresholds_st)
                df_with_signals = generate_signals(df.iloc[i:min(i+window,len(df))].copy(), indicator, results, thresholds_st)
            updated_df.loc[i , indicator + '_value'] = df_with_signals.loc[i%window, indicator]
            updated_df.loc[i, indicator + '_sign'] = df_with_signals.loc[i%window, 'final_signal']
            updated_df.loc[i, indicator + '_condition'] = df_with_signals.loc[i%window, 'condition']
    return updated_df

In [5]:
# 滚动窗口更新信号并应用
import os
# 设置文件路径
file_path = f'rolling_data_{contract}_{rolling_window}.csv'
# 如果文件不存在，运行滚动窗口更新信号并应用

rolling_signal = rolling_threshold_update(indicators_data, indicators, window=rolling_window)

    # 将结果保存到 CSV 文件
rolling_signal.to_csv(file_path, index=False)
print(f"滚动信号已生成并保存为 {file_path}")

滚动信号已生成并保存为 rolling_data_T_252.csv


In [6]:
rolling_signal

,date,close,high,low,open,volume,mom,cmo,apo,macd,...,d_condition,ri_value,ri_sign,ri_condition,cvi_value,cvi_sign,cvi_condition,obv_value,obv_sign,obv_condition
0,2015-03-20,97.090,97.960,96.530,96.900,3332.0,NaN,NaN,NaN,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-03-23,97.020,97.095,96.840,97.095,1386.0,NaN,-100.000000,NaN,-0.009697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-03-24,96.920,97.220,96.875,97.135,1380.0,NaN,-100.000000,NaN,-0.028621,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-03-25,96.850,97.030,96.760,97.020,1234.0,NaN,-100.000000,NaN,-0.047509,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-03-26,96.370,96.890,96.370,96.835,1073.0,NaN,-100.000000,NaN,-0.121764,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,2025-09-15,107.805,107.875,107.670,107.690,94600.0,-0.195,-13.588850,-0.02950,-0.006105,...,turn_warm,44.973545,1.0,cool,0.041072,1.0,cool,4678178.0,1.0,cool
2554,2025-09-16,108.000,108.085,107.580,107.680,138741.0,0.045,2.839117,-0.02225,0.040859,...,turn_warm,49.514563,1.0,cool,0.044359,1.0,cool,4816919.0,1.0,cool
2555,2025-09-17,108.155,108.165,107.915,107.985,101611.0,-0.005,-0.325733,-0.03775,0.090548,...,turn_warm,59.045226,1.0,turn_warm,0.043611,1.0,cool,4918530.0,1.0,cool
2556,2025-09-18,108.080,108.125,107.985,108.065,93558.0,-0.180,-11.920530,-0.05975,0.106406,...,turn_warm,56.585366,1.0,turn_warm,0.042592,1.0,cool,4824972.0,1.0,cool
